### 手写数字识别

In [ ]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from utils.DatasetUtils import DatasetUtils

data_utils= DatasetUtils()

# 加载数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
eval_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 查看数据集的样例

data_utils.show_image_simple(train_dataset,count=4)

In [ ]:
# 模型组网
from torchkeras import summary
from model.ClassifyNet import ClassifyNet

from utils.ModelWorker.ClassifyModelWorker import ClassifyModelWorker

input_shape=data_utils.get_data_shape(train_dataset)
model=ClassifyNet(
    input_shape=(1,28,28),
    num_classes=10
)

model_worker=ClassifyModelWorker(model)

print(f"{input_shape=}")
model_info=summary(model,input_shape=input_shape)

In [ ]:
# 模型准备
from torch.utils.data import DataLoader
from torch import nn
from torch import optim

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=64)

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# 模型训练

model_worker.train(
    criterion=loss,
    optimizer=optimizer,
    train_loader=train_loader,
    eval_loader=eval_loader,
    epochs=5,
)

In [ ]:
# 模型评估

model_worker.evaluate(
    eval_loader=eval_loader,
    criterion=loss,
)

In [ ]:
# 保存模型
model_worker.save('./output/HDR.pth')

In [ ]:
# 加载模型
model_worker.load('./output/HDR.pth')

In [ ]:
# 模型预测

indexs,inputs,labels = data_utils.select_simple(eval_dataset,count=4)
preds,acc = model_worker.predict(inputs=inputs,labels=labels)

print(f"{inputs.shape=}")
print(f"{acc=}")
print(f"indexs:{indexs}")
print(f"labels:{labels}")
print(f"preds :{preds}")

data_utils.show_image_batch(inputs,with_title=False)